# Análise comparativa

O objetivo dessa eanalise é comparar modelos para encontrar um que possa ser melhor utilizado dentro do problema em questão (estimar a idade por meio do numero de aneis), para isso antes realizamos a preparação e o pré-processamento dos dados.

## Preparação dos dados

### 1.1 Configurações iniciais

Importações e configurações

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_predict, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, train_test_split

from sklearn.model_selection import (
    cross_validate, GridSearchCV, ShuffleSplit
)

### 1.2 Obtendo dados

Nessa etapa vamos importar arquivos brutos de dados e o dicionário antes de começarmos o pre-processamento

In [3]:
#Importando dados
names = ['Sexo', 'Comprimento', 'Diâmetro', 'Altura', 'Peso total', 'Peso sem concha', 'Peso do intestino', 'Peso da concha', 'Anéis']
df = pd.read_csv("https://raw.githubusercontent.com/atlantico-academy/equipe-01/master/data/raw/abalone.csv", header=None, names=names)

In [4]:
df.head()

,Sexo,Comprimento,Diâmetro,Altura,Peso total,Peso sem concha,Peso do intestino,Peso da concha,Anéis
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


Obtendo informações sobre o conjunto de dados

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Sexo               4177 non-null   object 
 1   Comprimento        4177 non-null   float64
 2   Diâmetro           4177 non-null   float64
 3   Altura             4177 non-null   float64
 4   Peso total         4177 non-null   float64
 5   Peso sem concha    4177 non-null   float64
 6   Peso do intestino  4177 non-null   float64
 7   Peso da concha     4177 non-null   float64
 8   Anéis              4177 non-null   int64  
dtypes: float64(7), int64(1), object(1)
memory usage: 293.8+ KB


In [6]:
df.isna().sum()

Sexo                 0
Comprimento          0
Diâmetro             0
Altura               0
Peso total           0
Peso sem concha      0
Peso do intestino    0
Peso da concha       0
Anéis                0
dtype: int64

O conjunto de dados é composto de 8 variáveis de entrada (7 contínuas e 1 categórica) e 1 variável de saída (inteira). Ele não possui dados faltantes.

Definindo os parametros do dataset de entrada e saida

In [7]:
nominal_columns = ['Sexo']
continuos_columns = ['Comprimento', 'Diâmetro', 'Altura', 'Peso total', 'Peso sem concha', 'Peso do intestino', 'Peso da concha']
target_column = ['Anéis']

In [8]:
# Fizemos aqui a um drop (exclusão temporaria) da coluna aneis para iniciar o modelo entrada (oque a maquina vai interpretar)
# e saida( oque ela vai tentar adivinhar)
X = df.drop(df[target_column], axis=1)
y = df[target_column]

In [9]:
X.head()

,Sexo,Comprimento,Diâmetro,Altura,Peso total,Peso sem concha,Peso do intestino,Peso da concha
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055


In [10]:
y.head()

,Anéis
0,15
1,7
2,9
3,10
4,7


### 1.3 Tratamento de dados

Aqui realizamos a normalização e codificação de variáveis categóricas

#### 1.3.1 - Tratamento de dados discrepantes

Não realizamos, por ter feito testes e nenhum dado ter sido removido 

### 1.4 Codificação das variáveis categóricas e Normalização dos dados.

Iremos utilizar o metodo de normalização de dados chamado:

- Z-Score

#### Criando pipeline

In [11]:
nominal_preprocessor = Pipeline(steps=[
    # Codificação das variáveis
    ('encoding', OneHotEncoder())
])
continuous_preprocessor = Pipeline(steps=[
    # Normalização
    ('normalization', StandardScaler(with_mean=False))
])

preprocessor = ColumnTransformer([
    ('nominal', nominal_preprocessor, nominal_columns),
    ('continuos', continuous_preprocessor, continuos_columns)
])

## Escolha do modelo

### 2.1 Metodologia

Iremos análisar quatro modelos, que serão testados utilizando um método de validação cruzada por permutação, os modelos que serão testados serão:

- Regressão linear simples (OLS)
- Regressão Penalizada Ridge (RR)
- K-Nearest-Neighbors (KNN)
- Support Vector Machine (SVM)

Além disso, cada um desses algoritmos será testado com diferentes parametros, para que possamos encontrar o melhor modelo e a melhor configuração possível para esse modelo.

### 2.2 Modelos Lineares de Regressão

Explicar modelos

In [12]:
linear_models = [
    (
        "OLS",
        LinearRegression(),
        {
            
        }

    ),
    (
        "RR",
        Ridge(),
        {
            "alpha": np.arange(0.01, 1.0, 0.01)
        }
    )
]

Explicar parâmetros utilizados

In [16]:
n_splits = 5
final_results_lm = {}
for model_name, model, model_hparams_grid in linear_models:
    print(f"{model_name} is running...")
    model_gs = GridSearchCV(
        model,
        model_hparams_grid,
        scoring='neg_root_mean_squared_error',
        cv=5
    )
    approach = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("model", model_gs)
    ])
    results = cross_validate(
        approach,
        X=X,
        y = y.to_numpy().ravel(),
        scoring=[
            "neg_root_mean_squared_error",
            "r2"
        ],
        cv=ShuffleSplit(n_splits=n_splits, test_size=.2)
    )
    results["name"] = [model_name] * n_splits
    results["test_root_mean_squared_error"] = np.abs(results.pop("test_neg_root_mean_squared_error"))
    if final_results_lm:
        for key, value in results.items():
            final_results_lm[key] = np.append(final_results_lm[key], value)
    else:
        final_results_lm = results

OLS is running...
RR is running...


### 2.3 Modelos Não-Lineares de Regressão

Explicar modelos

In [18]:
nonlinear_models = [
    (
        "KNN",
        KNeighborsRegressor(),
        {
            "n_neighbors": np.arange(1, 50, 2),
            "weights": ["uniform","distance"]
        }
    ),
    
    (
        "SVR",
        SVR(max_iter=10000),
        {
            "kernel": ["rbf", "poly", "sigmoid"]
        }
    )
]

Explicar parâmetros

In [19]:
n_splits = 5
final_results_nlm = {}
for model_name, model, model_hparams_grid in nonlinear_models:
    print(f"{model_name} is running...")
    model_gs = GridSearchCV(
        model,
        model_hparams_grid,
        scoring='neg_root_mean_squared_error',
        cv=5
    )
    approach = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("model", model_gs)
    ])
    results = cross_validate(
        approach,
        X=X,
        y = y.to_numpy().ravel(),
        scoring=[
            "neg_root_mean_squared_error",
            "r2"
        ],
        cv=ShuffleSplit(n_splits=n_splits, test_size=.2)
    )
    results["name"] = [model_name] * n_splits
    results["test_root_mean_squared_error"] = np.abs(results.pop("test_neg_root_mean_squared_error"))
    if final_results_nlm:
        for key, value in results.items():
            final_results_nlm[key] = np.append(final_results_nlm[key], value)
    else:
        final_results_nlm = results

KNN is running...
SVR is running...


c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\U

### 2.4 Resultados da experimentação

#### 2.4.1 Modelos Lineares

In [21]:
(
    pd
    .DataFrame(final_results_lm)
    .groupby('name')
    .agg(['mean', 'std'])
    .transpose()
)

name                                    OLS        RR
fit_time                     mean  0.028263  0.839795
                             std   0.007580  0.069599
score_time                   mean  0.004105  0.003654
                             std   0.000784  0.000997
test_r2                      mean  0.527800  0.522905
                             std   0.019556  0.019068
test_root_mean_squared_error mean  2.171073  2.222376
                             std   0.040034  0.115766

#### 2.4.2 Modelos Não-Lineares

In [20]:
(
    pd
    .DataFrame(final_results_nlm)
    .groupby('name')
    .agg(['mean', 'std'])
    .transpose()
)

name                                    KNN       SVR
fit_time                     mean  6.177961  9.320571
                             std   0.676770  0.971586
score_time                   mean  0.033518  0.291088
                             std   0.011068  0.034869
test_r2                      mean  0.541106  0.536660
                             std   0.021689  0.020435
test_root_mean_squared_error mean  2.173335  2.212895
                             std   0.103400  0.093362

O melhor modelo obtido foi : KNN 